## 第一步：初始化 ROS 2 和导入库

## Step 1: 初始化 ROS 2

In [ ]:
import rclpy
from rclpy.node import Node
from sensor_msgs.msg import JointState
from std_msgs.msg import Header
import math
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import sys
import os

# 初始化 ROS 2
if not rclpy.ok():
    rclpy.init()
    print("✅ ROS 2 已初始化")
else:
    print("ℹ️ ROS 2 已经初始化")

## Step 2: 创建控制器类

In [ ]:
class PandaController(Node):
    def __init__(self):
        super().__init__('panda_interactive_controller')
        self.publisher = self.create_publisher(
            JointState,
            '/joint_states',
            10
        )
        
        self.joint_names = [
            'panda_joint1', 'panda_joint2', 'panda_joint3', 'panda_joint4',
            'panda_joint5', 'panda_joint6', 'panda_joint7', 'panda_hand_joint',
            'panda_leftfinger_joint', 'panda_rightfinger_joint'
        ]
        
        self.current_positions = {name: 0.0 for name in self.joint_names}
    
    def publish_joint_state(self, positions):
        msg = JointState()
        msg.header = Header()
        msg.header.stamp = self.get_clock().now().to_msg()
        msg.name = self.joint_names
        
        for name, value in positions.items():
            if name in self.current_positions:
                self.current_positions[name] = value
        
        msg.position = [self.current_positions[name] for name in self.joint_names]
        msg.velocity = [0.0] * len(self.joint_names)
        msg.effort = []
        
        self.publisher.publish(msg)
    
    def set_home(self):
        positions = {name: 0.0 for name in self.joint_names}
        self.publish_joint_state(positions)

controller = PandaController()
print("✅ 控制器已创建")

## Step 3: 创建交互式滑块

In [ ]:
sliders = {}
output_displays = {}

revolute_joints = [
    ('panda_joint1', -165, 165, 'Joint 1'),
    ('panda_joint2', -101, 101, 'Joint 2'),
    ('panda_joint3', -165, 165, 'Joint 3'),
    ('panda_joint4', -176, -4, 'Joint 4'),
    ('panda_joint5', -165, 165, 'Joint 5'),
    ('panda_joint6', -1, 215, 'Joint 6'),
    ('panda_joint7', -165, 165, 'Joint 7'),
    ('panda_hand_joint', 0, 2.3, 'Hand Joint'),
]

prismatic_joints = [
    ('panda_leftfinger_joint', 0.0, 0.04, 'Left Finger'),
    ('panda_rightfinger_joint', 0.0, 0.04, 'Right Finger'),
]

print("\n📊 旋转关节控制（度数）\n")
for joint_name, min_val, max_val, label in revolute_joints:
    slider = widgets.FloatSlider(
        value=0, min=min_val, max=max_val, step=1,
        description=label, orientation='horizontal',
        layout=widgets.Layout(width='100%')
    )
    output = widgets.HTML(value=f"<b>{joint_name}</b>: 0.0° | 0.000 rad")
    sliders[joint_name] = slider
    output_displays[joint_name] = output
    display(widgets.VBox([slider, output]))

print("\n📊 棱柱关节控制（米）\n")
for joint_name, min_val, max_val, label in prismatic_joints:
    slider = widgets.FloatSlider(
        value=0, min=min_val, max=max_val, step=0.001,
        description=label, orientation='horizontal',
        layout=widgets.Layout(width='100%')
    )
    output = widgets.HTML(value=f"<b>{joint_name}</b>: 0.000 m")
    sliders[joint_name] = slider
    output_displays[joint_name] = output
    display(widgets.VBox([slider, output]))

## Step 4: 绑定滑块事件

In [ ]:
def on_slider_change(change, joint_name, is_prismatic=False):
    value_deg = change['new']
    
    if is_prismatic:
        value_rad = value_deg
        output_text = f"<b>{joint_name}</b>: {value_deg:.3f} m"
    else:
        value_rad = math.radians(value_deg)
        output_text = f"<b>{joint_name}</b>: {value_deg:.1f}° | {value_rad:.3f} rad"
    
    output_displays[joint_name].value = output_text
    controller.publish_joint_state({joint_name: value_rad})

for joint_name, _, _, _ in revolute_joints:
    sliders[joint_name].observe(
        lambda change, jn=joint_name: on_slider_change(change, jn, is_prismatic=False),
        names='value'
    )

for joint_name, _, _, _ in prismatic_joints:
    sliders[joint_name].observe(
        lambda change, jn=joint_name: on_slider_change(change, jn, is_prismatic=True),
        names='value'
    )

print("✅ 滑块事件已绑定")

## Step 5: 预设位置按钮

In [ ]:
def update_sliders(positions):
    for joint_name, value in positions.items():
        if joint_name in sliders:
            is_prismatic = joint_name in ['panda_leftfinger_joint', 'panda_rightfinger_joint']
            sliders[joint_name].value = value if is_prismatic else math.degrees(value)

def on_home_clicked(b):
    controller.set_home()
    update_sliders({name: 0.0 for name in controller.joint_names})
    print("🏠 已设置为初始位置")

btn_home = widgets.Button(description='🏠 初始位置', button_style='info')
btn_home.on_click(on_home_clicked)

print("\n🎮 预设位置\n")
display(widgets.HBox([btn_home]))

## 测试单元格

In [ ]:
# 在这里写你的控制代码
print("修改这个单元格并运行你的代码来控制机器人")

In [ ]:
import rclpy
from sensor_msgs.msg import JointState
from std_msgs.msg import Header
import math
import ipywidgets as widgets
from IPython.display import display, HTML
import numpy as np

# 初始化 ROS 2
if not rclpy.ok():
    rclpy.init()

print("✓ ROS 2 已初始化")

## 第二步：创建 Panda 控制器类

In [ ]:
class PandaController:
    """Panda 机器人控制器"""
    
    def __init__(self):
        # 创建发布节点
        self.node = rclpy.create_node('panda_jupyter_controller')
        self.joint_pub = self.node.create_publisher(
            JointState, '/joint_states', 10
        )
        
        # 关节配置
        self.joint_names = [
            'panda_joint1',
            'panda_joint2', 
            'panda_joint3',
            'panda_joint4',
            'panda_joint5',
            'panda_joint6',
            'panda_joint7',
            'panda_hand_joint',
            'panda_leftfinger_joint',
            'panda_rightfinger_joint'
        ]
        
        # 关节角度限制（弧度）
        self.joint_limits = {
            'panda_joint1': (-2.8973, 2.8973),
            'panda_joint2': (-1.7628, 1.7628),
            'panda_joint3': (-2.8973, 2.8973),
            'panda_joint4': (-3.0718, -0.0698),
            'panda_joint5': (-2.8973, 2.8973),
            'panda_joint6': (-0.0175, 3.7525),
            'panda_joint7': (-2.8973, 2.8973),
            'panda_hand_joint': (0.0, 0.04),
            'panda_leftfinger_joint': (0.0, 0.04),
            'panda_rightfinger_joint': (0.0, 0.04),
        }
        
        # 当前关节角度
        self.current_angles = {name: 0.0 for name in self.joint_names}
        
        print(f"✓ Panda 控制器已初始化")
        print(f"✓ 可控制的关节数: {len(self.joint_names)}")
    
    def publish_state(self, angles_dict=None):
        """发布关节状态"""
        if angles_dict:
            self.current_angles.update(angles_dict)
        
        msg = JointState()
        msg.header = Header()
        msg.header.stamp = self.node.get_clock().now().to_msg()
        msg.name = self.joint_names
        msg.position = [self.current_angles[name] for name in self.joint_names]
        msg.velocity = [0.0] * len(self.joint_names)
        msg.effort = [0.0] * len(self.joint_names)
        
        self.joint_pub.publish(msg)
        rclpy.spin_once(self.node, timeout_sec=0.01)

# 创建全局控制器实例
controller = PandaController()
print("✓ 准备就绪！")

## 第三步：预设位置

In [ ]:
# 预设位置按钮
def set_home():
    """设置为初始位置"""
    angles = {name: 0.0 for name in controller.joint_names}
    controller.publish_state(angles)
    print("✓ 已设置为初始位置")

def set_ready():
    """设置为准备位置"""
    angles = {
        'panda_joint1': 0.0,
        'panda_joint2': -math.pi / 4,
        'panda_joint3': 0.0,
        'panda_joint4': -3 * math.pi / 4,
        'panda_joint5': 0.0,
        'panda_joint6': math.pi / 2,
        'panda_joint7': math.pi / 4,
        'panda_hand_joint': 0.0,
        'panda_leftfinger_joint': 0.0,
        'panda_rightfinger_joint': 0.0,
    }
    controller.publish_state(angles)
    print("✓ 已设置为准备位置")

def set_stretch():
    """设置为伸展位置"""
    angles = {
        'panda_joint1': 0.0,
        'panda_joint2': -math.pi / 6,
        'panda_joint3': 0.0,
        'panda_joint4': -math.pi / 2,
        'panda_joint5': 0.0,
        'panda_joint6': math.pi / 3,
        'panda_joint7': 0.0,
        'panda_hand_joint': 0.0,
        'panda_leftfinger_joint': 0.0,
        'panda_rightfinger_joint': 0.0,
    }
    controller.publish_state(angles)
    print("✓ 已设置为伸展位置")

# 创建按钮
btn_home = widgets.Button(description='初始位置', button_style='info')
btn_ready = widgets.Button(description='准备位置', button_style='success')
btn_stretch = widgets.Button(description='伸展位置', button_style='warning')

btn_home.on_click(lambda x: set_home())
btn_ready.on_click(lambda x: set_ready())
btn_stretch.on_click(lambda x: set_stretch())

display(widgets.HBox([btn_home, btn_ready, btn_stretch]))

## 第四步：关节控制滑块

In [ ]:
# 创建关节滑块
sliders = {}
outputs = {}

for joint_name in controller.joint_names:
    min_val, max_val = controller.joint_limits[joint_name]
    
    # 将限制转换为度数以便显示
    min_deg = math.degrees(min_val)
    max_deg = math.degrees(max_val)
    
    slider = widgets.FloatSlider(
        value=0.0,
        min=min_deg,
        max=max_deg,
        step=1,
        description=joint_name,
        style={'description_width': '180px'},
        layout=widgets.Layout(width='500px')
    )
    
    output = widgets.Output()
    
    def make_on_change(jname):
        def on_change(change):
            # 将度数转换为弧度
            angle_rad = math.radians(change['new'])
            controller.publish_state({jname: angle_rad})
            
            with output:
                output.clear_output(wait=True)
                print(f"角度: {change['new']:.1f}° ({angle_rad:.3f} rad)")
        return on_change
    
    slider.observe(make_on_change(joint_name), names='value')
    sliders[joint_name] = slider
    outputs[joint_name] = output

# 显示滑块
print("\n🎮 关节控制滑块")
print("="*70)
for joint_name in controller.joint_names:
    display(widgets.VBox([sliders[joint_name], outputs[joint_name]]))

## 第五步：夹爪控制

In [ ]:
def open_gripper():
    angles = {
        'panda_leftfinger_joint': 0.04,
        'panda_rightfinger_joint': 0.04,
    }
    controller.publish_state(angles)
    print("✓ 夹爪已打开")

def close_gripper():
    angles = {
        'panda_leftfinger_joint': 0.0,
        'panda_rightfinger_joint': 0.0,
    }
    controller.publish_state(angles)
    print("✓ 夹爪已关闭")

# 创建夹爪控制按钮
btn_open = widgets.Button(description='打开夹爪', button_style='success')
btn_close = widgets.Button(description='关闭夹爪', button_style='danger')

btn_open.on_click(lambda x: open_gripper())
btn_close.on_click(lambda x: close_gripper())

print("\n🤲 夹爪控制")
print("="*70)
display(widgets.HBox([btn_open, btn_close]))

## 使用说明

### 方式 1：使用预设位置
- 点击上面的按钮快速设置机器人到特定位置

### 方式 2：使用关节滑块
- 拖动滑块改变单个关节的角度
- 角度范围显示在滑块上（度数）
- 实时看到机器人在 RViz2 中的运动

### 方式 3：夹爪控制
- 使用打开/关闭按钮控制夹爪

### 关节限制
- 每个关节都有物理限制（在滑块上显示）
- 超出限制时滑块会自动限制在有效范围内

## 高级用法：在 Python 代码中控制

你也可以直接在代码中精确控制关节角度：

In [ ]:
# 示例：设置特定的关节角度
example_angles = {
    'panda_joint1': math.radians(45),   # 45 度
    'panda_joint2': math.radians(-30),  # -30 度
    'panda_joint3': math.radians(0),    # 0 度
}

# 取消注释下面的行来执行
# controller.publish_state(example_angles)
# print("✓ 已发布自定义角度")

print("示例代码已准备好，取消注释上面的行来执行")

## 清理

完成后运行此单元格清理资源：

In [ ]:
# 清理
# controller.node.destroy_node()
# rclpy.shutdown()
# print("✓ 已清理资源")

print("清理代码已准备好，取消注释上面的行来执行")